In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import sys
from collections import Counter
import os

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return np.nan # 모든 y_true가 0인 경우 NaN 반환
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# --- 0. 파일 경로 설정 ---
review_json_path = 'review.json'
absa_results_path = 'absa_atepc_results.json' # .json 확장자로 변경

# --- 1. 데이터 로드 및 전처리 ---
print("--- 1단계: 데이터 로드 및 전처리 시작 ---")
try:
    data = []
    with open(review_json_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    print(f"'{len(df)}'개의 리뷰 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{review_json_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()
except json.JSONDecodeError as e:
    print(f"오류: '{review_json_path}' 파일 파싱 중 오류 발생: {e}. 파일 내용이 올바른 JSON 형식을 따르는지 확인해주세요.")
    sys.exit()

user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df['user_idx'] = user_encoder.fit_transform(df['user_id'])
df['business_idx'] = business_encoder.fit_transform(df['business_id'])
num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)
print(f"'{num_users}'명의 고유 사용자와 '{num_businesses}'개의 고유 레스토랑을 매핑했습니다.")

try:
    df['date'] = pd.to_datetime(df['date'], unit='ms') # timestamp in milliseconds
except ValueError:
    df['date'] = pd.to_datetime(df['date']) # assume standard date string format if ms fails

df = df.sort_values(by='date').reset_index(drop=True)
total_reviews = len(df)
train_split = int(total_reviews * 0.7)
val_split_idx = int(total_reviews * (0.7 + 0.1)) # 검증 세트의 끝 인덱스
train_df = df.iloc[:train_split].copy()
val_df = df.iloc[train_split:val_split_idx].copy()
test_df = df.iloc[val_split_idx:].copy() # 나머지 20%를 테스트 세트로
print(f"학습 세트 크기: {len(train_df)}개 리뷰")
print(f"검증 세트 크기: {len(val_df)}개 리뷰")
print(f"테스트 세트 크기: {len(test_df)}개 리뷰")

print("\n--- 2단계: absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 시작 ---")

absa_data = {}
try:
    with open(absa_results_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            absa_data[entry['review_id']] = entry['aspects']
    print(f"'{absa_results_path}' 파일에서 {len(absa_data)}개의 리뷰 속성 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: '{absa_results_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    sys.exit()
except json.JSONDecodeError as e:
    print(f"오류: '{absa_results_path}' 파일 파싱 중 오류 발생: {e}. 파일 내용이 올바른 JSON 형식을 따르는지 확인해주세요.")
    sys.exit()

all_extracted_aspect_terms_from_file = []
all_extracted_sentiments_from_file = []

for review_id in train_df['review_id']:
    aspects = absa_data.get(review_id, [])
    if not aspects: # 속성이 없는 경우 'general' 및 'neutral'로 대체
        all_extracted_aspect_terms_from_file.append('general')
        all_extracted_sentiments_from_file.append('neutral')
    else:
        for aspect in aspects:
            if 'term' in aspect and 'sentiment' in aspect:
                all_extracted_aspect_terms_from_file.append(aspect['term'].lower())
                # 감성 라벨을 소문자로 통일 (Positive, Negative, Neutral)
                all_extracted_sentiments_from_file.append(aspect['sentiment'].lower())

# 속성 키워드 어휘집 구축
term_counts = Counter(all_extracted_aspect_terms_from_file)
# <PAD>는 0번, <UNK>는 1번, 'general'은 2번 인덱스를 가집니다.
ASPECT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'general': 2}
next_id = 3
MIN_TERM_FREQUENCY = 1 # 어휘집 구축 시 최소 등장 빈도
filtered_terms = [term for term, count in term_counts.items() if count >= MIN_TERM_FREQUENCY]
filtered_terms.sort(key=lambda x: term_counts[x], reverse=True)

for term in filtered_terms:
    if term not in ASPECT_TO_ID:
        ASPECT_TO_ID[term] = next_id
        next_id += 1
NUM_ASPECT_TERMS = len(ASPECT_TO_ID)

# 감성 어휘집 구축 (고정된 긍정, 부정, 중립)
SENTIMENT_TO_ID = {'<PAD>': 0, '<UNK>': 1, 'positive': 2, 'negative': 3, 'neutral': 4}
# PyABSA 결과에 따라 'Positive', 'Negative', 'Neutral'을 소문자로 통일하여 처리
NUM_SENTIMENTS = len(SENTIMENT_TO_ID)

print(f"동적으로 구축된 속성 어휘집 크기 (학습 데이터 기반): {NUM_ASPECT_TERMS}개 키워드")
print(f"어휘집 상위 10개 키워드: {list(ASPECT_TO_ID.items())[3:13]}...") # 0,1,2 제외하고 출력
print(f"구축된 감성 어휘집 크기: {NUM_SENTIMENTS}개 감성 ({list(SENTIMENT_TO_ID.items())})")
print("absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 완료.")

print("\n--- 2.5단계: 로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 시작 ---")

def map_aspect_and_sentiment_ids_from_file(df_subset, absa_data_map, aspect_to_id_map, sentiment_to_id_map):
    extracted_aspect_ids_list = []
    extracted_sentiment_ids_list = []
    
    for _, row in df_subset.iterrows():
        review_id = row['review_id']
        aspects_for_review = absa_data_map.get(review_id, [])
        
        aspect_ids_for_review = []
        sentiment_ids_for_review = []
        
        if aspects_for_review:
            for aspect in aspects_for_review:
                if 'term' in aspect and 'sentiment' in aspect:
                    aspect_id = aspect_to_id_map.get(aspect['term'].lower(), aspect_to_id_map['<UNK>'])
                    sentiment_id = sentiment_to_id_map.get(aspect['sentiment'].lower(), sentiment_to_id_map['<UNK>']) # 감성 ID도 매핑
                    
                    aspect_ids_for_review.append(aspect_id)
                    sentiment_ids_for_review.append(sentiment_id)
        
        if not aspect_ids_for_review: # 추출된 속성이 없으면 'general' 및 'neutral' 감성 사용
            aspect_ids_for_review.append(aspect_to_id_map['general'])
            sentiment_ids_for_review.append(sentiment_to_id_map['neutral']) # 'general'과 함께 'neutral' 감성 할당
            
        extracted_aspect_ids_list.append(aspect_ids_for_review)
        extracted_sentiment_ids_list.append(sentiment_ids_for_review)
        
    return extracted_aspect_ids_list, extracted_sentiment_ids_list

# 각 DataFrame에 속성 ID와 감성 ID 리스트 매핑
train_df['extracted_aspect_ids'], train_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(train_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)
val_df['extracted_aspect_ids'], val_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(val_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)
test_df['extracted_aspect_ids'], test_df['extracted_sentiment_ids'] = map_aspect_and_sentiment_ids_from_file(test_df, absa_data, ASPECT_TO_ID, SENTIMENT_TO_ID)

print("로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 완료.")

class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_indices = torch.tensor(df['user_idx'].values, dtype=torch.long)
        self.business_indices = torch.tensor(df['business_idx'].values, dtype=torch.long)
        self.extracted_aspect_ids = df['extracted_aspect_ids'].values.tolist()
        self.extracted_sentiment_ids = df['extracted_sentiment_ids'].values.tolist() # 감성 ID 추가
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)
    def __len__(self):
        return len(self.stars)
    def __getitem__(self, idx):
        return (self.user_indices[idx], self.business_indices[idx],
                self.extracted_aspect_ids[idx], self.extracted_sentiment_ids[idx], # 감성 ID 반환
                self.stars[idx])

def custom_collate_fn(batch):
    user_indices, business_indices, extracted_aspect_ids_list, extracted_sentiment_ids_list, stars = zip(*batch)

    user_indices = torch.stack(user_indices)
    business_indices = torch.stack(business_indices)
    stars = torch.stack(stars)

    # MultiheadAttention 입력을 위한 패딩 및 마스크 생성
    max_aspect_len = max(len(ids) for ids in extracted_aspect_ids_list)
    
    padded_aspect_ids = [ids + [ASPECT_TO_ID['<PAD>']] * (max_aspect_len - len(ids)) for ids in extracted_aspect_ids_list]
    padded_aspect_ids_tensor = torch.tensor(padded_aspect_ids, dtype=torch.long)
    
    padded_sentiment_ids = [ids + [SENTIMENT_TO_ID['<PAD>']] * (max_aspect_len - len(ids)) for ids in extracted_sentiment_ids_list]
    padded_sentiment_ids_tensor = torch.tensor(padded_sentiment_ids, dtype=torch.long)
    
    # key_padding_mask: True는 무시할 요소 (패딩)
    attn_mask = (padded_aspect_ids_tensor == ASPECT_TO_ID['<PAD>']) # aspect_ids를 기준으로 마스크 생성

    return user_indices, business_indices, padded_aspect_ids_tensor, padded_sentiment_ids_tensor, stars, attn_mask

print("\n--- 3단계: AAT-Rec 모델 구성 요소 정의 (감성 통합) ---")

class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims, dropout_rate):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        layers = []
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim

    def forward(self, user_indices, business_indices):
        user_emb = self.user_embedding(user_indices)
        business_emb = self.business_embedding(business_indices)
        combined_emb = torch.cat((user_emb, business_emb), dim=-1)
        interaction_features = self.mlp(combined_emb)
        return interaction_features

# 리뷰 속성별 키워드 추출 모듈 (MultiheadAttention 사용) - 감성 정보 통합
class ReviewAspectTermExtractionModule(nn.Module):
    def __init__(self, num_aspect_terms, aspect_embedding_dim, num_sentiments, sentiment_embedding_dim, num_heads, mlp_dims, dropout_rate):
        super(ReviewAspectTermExtractionModule, self).__init__()
        self.aspect_embedding = nn.Embedding(num_aspect_terms, aspect_embedding_dim, padding_idx=ASPECT_TO_ID['<PAD>'])
        self.sentiment_embedding = nn.Embedding(num_sentiments, sentiment_embedding_dim, padding_idx=SENTIMENT_TO_ID['<PAD>']) # 감성 임베딩 추가
        
        # MultiheadAttention의 embed_dim은 이제 속성 임베딩과 감성 임베딩을 합친 차원이 됩니다.
        combined_embedding_dim = aspect_embedding_dim + sentiment_embedding_dim
        self.multihead_attn = nn.MultiheadAttention(embed_dim=combined_embedding_dim, num_heads=num_heads, batch_first=True)
        
        layers = []
        input_dim = combined_embedding_dim # MultiheadAttention 출력 차원은 combined_embedding_dim과 동일
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = input_dim
            
    def forward(self, pre_extracted_aspect_ids_batch, pre_extracted_sentiment_ids_batch, attn_mask):
        embedded_terms = self.aspect_embedding(pre_extracted_aspect_ids_batch)
        embedded_sentiments = self.sentiment_embedding(pre_extracted_sentiment_ids_batch)
        
        # 속성 임베딩과 감성 임베딩을 결합 (Concatenate)
        combined_embeddings = torch.cat((embedded_terms, embedded_sentiments), dim=-1) # dim=-1은 마지막 차원 기준으로 합침
        
        # MultiheadAttention에 key_padding_mask 적용
        attn_output, _ = self.multihead_attn(
            combined_embeddings, 
            combined_embeddings, 
            combined_embeddings, 
            key_padding_mask=attn_mask
        )

        # 패딩된 요소들을 0으로 만들고 유효한 요소들만 평균 풀링
        valid_attn_output = attn_output * (~attn_mask.unsqueeze(-1)).float()
        pooled_output = valid_attn_output.sum(dim=1) / (~attn_mask).sum(dim=1, keepdim=True).float().clamp(min=1)
        
        aspect_features = self.mlp(pooled_output)
        return aspect_features

class RatingPredictionModule(nn.Module):
    def __init__(self, input_dim, mlp_dims, dropout_rate):
        super(RatingPredictionModule, self).__init__()
        layers = []
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1))
        self.mlp = nn.Sequential(*layers)
    def forward(self, combined_features):
        raw_prediction = self.mlp(combined_features)
        prediction = torch.sigmoid(raw_prediction) * 4 + 1 # 1~5점 스케일로 정규화
        return prediction

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, num_aspect_terms, aspect_embedding_dim,
                 num_sentiments, sentiment_embedding_dim, # 감성 관련 파라미터 추가
                 num_attn_heads, aspect_mlp_dims, final_mlp_dims, dropout_rate):
        super(AATRec, self).__init__()
        self.customer_restaurant_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims, dropout_rate
        )
        self.aspect_extraction_module = ReviewAspectTermExtractionModule(
            num_aspect_terms, aspect_embedding_dim, num_sentiments, sentiment_embedding_dim, # 감성 관련 파라미터 전달
            num_attn_heads, aspect_mlp_dims, dropout_rate
        )
        combined_feature_dim = self.customer_restaurant_module.output_dim + \
                               self.aspect_extraction_module.output_dim
        self.rating_prediction_module = RatingPredictionModule(
            combined_feature_dim, final_mlp_dims, dropout_rate
        )
    def forward(self, user_indices, business_indices, pre_extracted_aspect_ids, pre_extracted_sentiment_ids, attn_mask): # 감성 ID 추가
        interaction_features = self.customer_restaurant_module(user_indices, business_indices)
        aspect_features = self.aspect_extraction_module(pre_extracted_aspect_ids, pre_extracted_sentiment_ids, attn_mask) # 감성 ID 전달
        combined_features = torch.cat((interaction_features, aspect_features), dim=-1)
        predicted_rating = self.rating_prediction_module(combined_features).squeeze()
        return predicted_rating

print("AATRec 모델 구성 요소 정의 완료.")


# --- 4. 하이퍼파라미터 설정 및 모델 학습 ---

print("\n--- 4단계: 단일 하이퍼파라미터 조합으로 모델 학습 및 저장 시작 ---")

# Dataset 인스턴스 생성
train_dataset = ReviewDataset(train_df)
val_dataset = ReviewDataset(val_df)
test_dataset = ReviewDataset(test_df) # 테스트는 학습 후 최종 평가에만 사용

# 학습에 사용할 단일 하이퍼파라미터 조합 설정
best_params = {
    'embedding_dim': 64,
    'aspect_embedding_dim': 96, # 속성 임베딩 차원
    'sentiment_embedding_dim': 32, # 감성 임베딩 차원 (새로 추가)
    'num_attn_heads': 8,
    'learning_rate': 0.001,
    'batch_size': 128,
    'dropout_rate': 0.2,
    'user_biz_mlp_dims': [128, 64],
    'aspect_mlp_dims': [64, 32],
    'final_mlp_dims': [32, 16]
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"학습 장치: '{device}'")

best_overall_rmse = float('inf') # 최적 모델 저장을 위한 초기화
best_model_path = 'final_best_aatrec_model_multihead_aspect_sentiment.pt' # 최종 최적 모델 저장 경로 (이름 변경)

print(f"현재 파라미터: {best_params}")

# 현재 조합의 하이퍼파라미터 설정
EMBEDDING_DIM = best_params['embedding_dim']
ASPECT_EMBEDDING_DIM = best_params['aspect_embedding_dim']
SENTIMENT_EMBEDDING_DIM = best_params['sentiment_embedding_dim'] # 새로운 파라미터
NUM_ATTN_HEADS = best_params['num_attn_heads']
LEARNING_RATE = best_params['learning_rate']
BATCH_SIZE = best_params['batch_size']
DROPOUT_RATE = best_params['dropout_rate']
USER_BIZ_MLP_DIMS = best_params['user_biz_mlp_dims']
ASPECT_MLP_DIMS = best_params['aspect_mlp_dims']
FINAL_MLP_DIMS = best_params['final_mlp_dims']

# DataLoader 생성 (custom_collate_fn 다시 사용)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn)

model = AATRec(
    num_users, num_businesses, EMBEDDING_DIM,
    USER_BIZ_MLP_DIMS, NUM_ASPECT_TERMS, ASPECT_EMBEDDING_DIM,
    NUM_SENTIMENTS, SENTIMENT_EMBEDDING_DIM, # 감성 관련 파라미터 전달
    NUM_ATTN_HEADS, ASPECT_MLP_DIMS, FINAL_MLP_DIMS, DROPOUT_RATE
)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 조기 종료 설정
PATIENCE = 5 # 조기 종료 대기 횟수
MIN_DELTA = 0.001 # 검증 RMSE 개선을 위한 최소 변화량
current_best_val_rmse = float('inf') # 'current_best_val_rmse' 초기화
epochs_no_improve = 0

NUM_EPOCHS_PER_COMBINATION = 50 # 최대 에포크

print("\n--- 모델 학습 시작 ---")
for epoch in range(NUM_EPOCHS_PER_COMBINATION):
    model.train()
    total_train_loss = 0
    for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in train_loader: # 감성 ID 추가
        user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
            user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device) # 감성 ID to device
        attn_mask = attn_mask.to(device)
        
        optimizer.zero_grad()
        predicted_stars = model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask) # 감성 ID 전달
        loss = criterion(predicted_stars, stars)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)

    model.eval()
    total_val_loss = 0
    val_preds = []
    val_true = []
    with torch.no_grad():
        for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in val_loader: # 감성 ID 추가
            user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
                user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device) # 감성 ID to device
            attn_mask = attn_mask.to(device)
            predicted_stars = model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask) # 감성 ID 전달
            total_val_loss += criterion(predicted_stars, stars).item()
            val_preds.extend(predicted_stars.cpu().numpy())
            val_true.extend(stars.cpu().numpy())
    avg_val_loss = total_val_loss / len(val_loader)
    
    # 모델 출력에서 1~5점 스케일링이 이미 되어 있으므로, 추가 클리핑은 선택 사항이지만 안전을 위해 유지
    val_preds_clipped = np.clip(val_preds, 1.0, 5.0)
    current_val_rmse = np.sqrt(mean_squared_error(val_true, val_preds_clipped))

    sys.stdout.write(f"\r  Epoch {epoch+1}/{NUM_EPOCHS_PER_COMBINATION}, "
                     f"Train Loss: {avg_train_loss:.4f}, "
                     f"Val Loss: {avg_val_loss:.4f}, "
                     f"Val RMSE: {current_val_rmse:.4f}")
    sys.stdout.flush()

    if current_val_rmse < current_best_val_rmse - MIN_DELTA:
        current_best_val_rmse = current_val_rmse
        epochs_no_improve = 0
        torch.save(model.state_dict(), best_model_path) # 최적 모델 저장
        sys.stdout.write(f" --> 최적 검증 RMSE 개선됨: {current_best_val_rmse:.4f}. 모델 저장됨.\n")
        sys.stdout.flush()
    else:
        epochs_no_improve += 1
        if epochs_no_improve == PATIENCE:
            sys.stdout.write(f" --> 조기 종료! {PATIENCE} 에포크 동안 검증 RMSE 개선이 없었습니다.\n")
            sys.stdout.flush()
            break
            
print("\n--- 모델 학습 완료 ---")
print(f"최종 저장된 모델의 검증 RMSE: {current_best_val_rmse:.4f}")
print(f"저장 경로: {best_model_path}")


# --- 최종 모델 테스트 ---
print("\n--- 최종 모델 테스트 시작 (최적 파라미터 사용) ---")

if os.path.exists(best_model_path) and best_params:
    # 최적 파라미터로 모델 재구성 (저장된 모델 로드)
    final_model = AATRec(
        num_users, num_businesses, best_params['embedding_dim'],
        best_params['user_biz_mlp_dims'], NUM_ASPECT_TERMS, best_params['aspect_embedding_dim'],
        NUM_SENTIMENTS, best_params['sentiment_embedding_dim'], # 감성 관련 파라미터 전달
        best_params['num_attn_heads'], best_params['aspect_mlp_dims'], best_params['final_mlp_dims'], best_params['dropout_rate']
    )
    final_model.load_state_dict(torch.load(best_model_path, map_location=device)) # map_location 추가
    final_model.to(device)
    print(f"최적의 모델 가중치 '{best_model_path}' 로드 완료.")
else:
    print("경고: 최적의 모델 가중치를 찾을 수 없거나 최적 파라미터가 설정되지 않았습니다. 테스트를 건너뜜.")
    sys.exit()

# 테스트 로더도 custom_collate_fn 사용
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False, collate_fn=custom_collate_fn)

final_model.eval()
test_preds = []
test_true = []
with torch.no_grad():
    for user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars, attn_mask in test_loader: # 감성 ID 추가
        user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, stars = \
            user_indices.to(device), business_indices.to(device), extracted_aspect_ids.to(device), extracted_sentiment_ids.to(device), stars.to(device) # 감성 ID to device
        attn_mask = attn_mask.to(device)
        predicted_stars = final_model(user_indices, business_indices, extracted_aspect_ids, extracted_sentiment_ids, attn_mask) # 감성 ID 전달
        test_preds.extend(predicted_stars.cpu().numpy())
        test_true.extend(stars.cpu().numpy())

# 예측 범위 클리핑 (1점 ~ 5점)
test_preds_clipped = np.clip(test_preds, 1.0, 5.0)

mse = mean_squared_error(test_true, test_preds_clipped)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true, test_preds_clipped)
mape = mean_absolute_percentage_error(test_true, test_preds_clipped)

print(f"\n--- 최종 모델 성능 평가 (최적 파라미터) ---")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"사용된 최적 하이퍼파라미터: {best_params}")

--- 1단계: 데이터 로드 및 전처리 시작 ---
'447796'개의 리뷰 데이터를 성공적으로 로드했습니다.
'27807'명의 고유 사용자와 '6831'개의 고유 레스토랑을 매핑했습니다.
학습 세트 크기: 313457개 리뷰
검증 세트 크기: 44779개 리뷰
테스트 세트 크기: 89560개 리뷰

--- 2단계: absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 시작 ---
'absa_atepc_results.json' 파일에서 447796개의 리뷰 속성 데이터를 성공적으로 로드했습니다.
동적으로 구축된 속성 어휘집 크기 (학습 데이터 기반): 39160개 키워드
어휘집 상위 10개 키워드: [('food', 3), ('service', 4), ('staff', 5), ('place', 6), ('atmosphere', 7), ('prices', 8), ('they', 9), ('price', 10), ('pizza', 11), ('it', 12)]...
구축된 감성 어휘집 크기: 5개 감성 ([('<PAD>', 0), ('<UNK>', 1), ('positive', 2), ('negative', 3), ('neutral', 4)])
absa_atepc_results.json 파일에서 속성 데이터 로드 및 동적 어휘집 구축 완료.

--- 2.5단계: 로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 시작 ---
로드된 속성 데이터를 DataFrame에 매핑 및 캐싱 완료.

--- 3단계: AAT-Rec 모델 구성 요소 정의 (감성 통합) ---
AATRec 모델 구성 요소 정의 완료.

--- 4단계: 단일 하이퍼파라미터 조합으로 모델 학습 및 저장 시작 ---
학습 장치: 'cuda'
현재 파라미터: {'embedding_dim': 64, 'aspect_embedding_dim': 96, 'sentiment_embedding_dim': 32, 'num_attn_heads': 8, 'learning_rate